### Обучение SGDClassifier и NeighborSampler для предсказания 2 значной послеовательности (Код товарной позиции)

In [ ]:
import sklearn
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
import json
import pickle
import pandas as pd

In [2]:
def softmax(x):
  #создание вероятностного распределения
  proba = np.exp(-x)
  return proba / sum(proba)

In [3]:
class NeighborSampler(BaseEstimator):
  def __init__(self, k=5, temperature=10.0):
    self.k=k
    self.temperature = temperature
  def fit(self, X, y):
    self.tree_ = BallTree(X)
    self.y_ = np.array(y)
  def predict(self, X, random_state=None):
    distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
    result = []
    resultDist = []
    for distance, index in zip(distances, indices):
      result.append(np.random.choice(index, p=softmax(distance * self.temperature)))
      resultDist.append(np.random.choice(distance, p=softmax(distance * self.temperature)))
    return self.y_[result] , resultDist

In [4]:
data = pd.read_csv('dataset.csv',sep=';')

In [5]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
# Получение текстовой строки из списка слов
def str_corpus(corpus):
    str_corpus = ''
    for i in corpus:
        str_corpus += ' ' + i
    str_corpus = str_corpus.strip()
    return str_corpus
# Получение списка всех слов в корпусе
def get_corpus(data):
    corpus = []
    for phrase in data:
        for word in phrase.split():
            corpus.append(word)
    return corpus
# Получение облака слов
def get_wordCloud(corpus):
    wordCloud = WordCloud(background_color='white',
                              stopwords=STOPWORDS,
                              width=3000,
                              height=2500,
                              max_words=200,
                              random_state=42
                         ).generate(str_corpus(corpus))
    return wordCloud

In [7]:
data=data[:10000]

In [8]:
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

data['Post_clean'] = data['OPISANIE'].map(lambda x: x.lower())
data['Post_clean'] = data['Post_clean'].map(lambda x: remove_punct(x))
data['Post_clean'] = data['Post_clean'].map(lambda x: x.split(' '))
data['Post_clean'] = data['Post_clean'].map(lambda x: [token for token in x if token not in russian_stopwords\
                                                                  and token != " " \
                                                                  and token.strip() not in punctuation])
data['Post_clean'] = data['Post_clean'].map(lambda x: ' '.join(x))

In [9]:
data['TNVED2']=data['TNVED'].map(lambda x: int(str(x)[:-2]))

In [10]:
data['TNVED4']=data['TNVED'].map(lambda x: int(str(x)[2:]))

In [11]:
data.isnull().sum()

TNVED         0
OPISANIE      0
DATA          0
Post_clean    0
TNVED2        0
TNVED4        0
dtype: int64

In [12]:
data

,TNVED,OPISANIE,DATA,Post_clean,TNVED2,TNVED4
0,9025,"ЧАСТИ И ПРИНАДЛЕЖНОСТИ АППАРАТУРЫ ПОЗ.9025, НЕ...",07.09.2021,части принадлежности аппаратуры поз 9025 военн...,90,25
1,8504,"ТРАНСФОРМАТОРЫ, МОЩНОСТЬЮ БОЛЕЕ 1 КВА, НО НЕ Б...",07.09.2021,трансформаторы мощностью 1 ква 16 ква лом элек...,85,4
2,8481,"КЛАПАНЫ ЗАПОРНЫЕ ИЗ СТАЛИ, НЕ ВОЕННОГО НАЗНАЧЕ...",07.09.2021,клапаны запорные стали военного назначения,84,81
3,8531,УСТРОЙСТВА СИГНАЛИЗАЦИОННЫЕ ОХРАННЫЕ И УСТРОЙС...,07.09.2021,устройства сигнализационные охранные устройств...,85,31
4,3402,"ВЕЩЕСТВА ПОВЕРХНОСТНО-АКТИВНЫЕ ОРГАНИЧЕСКИЕ, А...",02.09.2021,вещества поверхностно активные органические ан...,34,2
...,...,...,...,...,...,...
9995,8512,"ПРИБОРЫ ОСВЕЩЕНИЯ И ВИЗУАЛЬНОЙ СИГНАЛИЗАЦИИ, Д...",03.09.2021,приборы освещения визуальной сигнализации м ni...,85,12
9996,8512,"ПРИБОРЫ ОСВЕЩЕНИЯ И ВИЗУАЛЬНОЙ СИГНАЛИЗАЦИИ, Д...",03.09.2021,приборы освещения визуальной сигнализации м ni...,85,12
9997,8512,"ПРИБОРЫ ОСВЕЩЕНИЯ И ВИЗУАЛЬНОЙ СИГНАЛИЗАЦИИ, Д...",03.09.2021,приборы освещения визуальной сигнализации м ni...,85,12
9998,8512,"ПРИБОРЫ ОСВЕЩЕНИЯ И ВИЗУАЛЬНОЙ СИГНАЛИЗАЦИИ, Д...",03.09.2021,приборы освещения визуальной сигнализации м ni...,85,12


In [13]:
TwoClasses = data['TNVED2'].unique()

In [103]:
import pymorphy2
Errors = []
morph = pymorphy2.MorphAnalyzer(lang='ru')
for TNVED in TwoClasses:
   try:
    df = data[data['TNVED2']==TNVED]
    questions_response = df['Post_clean'].tolist()
    answer_response = df['TNVED4'].tolist()
    questions = []
    answer = []
    transform=0
    for i in range(len(questions_response)):
          if questions_response[i]>"":
           phrases=questions_response[i]
   
            # разбираем вопрос на слова
           words=phrases.split(' ')
           phrase=""
           for word in words:
                word = morph.parse(word)[0].normal_form  
                phrase = phrase + word + " "
                # Если длинна полученной фразы больше 0 добавляем ей в массив вопросов и массив кодов ответов
           if (len(phrase)>0):
             questions.append(phrase.strip())
             transform=transform+1
             answer.append(answer_response[i])
    vectorizer_q = TfidfVectorizer()
    vectorizer_q.fit(questions)
    matrix_big_q = vectorizer_q.transform(questions)
    if transform>200:
              transform=200
    svd_q = TruncatedSVD(n_components=transform)
    svd_q.fit(matrix_big_q)
    matrix_small_q = svd_q.transform(matrix_big_q)
    ns_q = NeighborSampler()
    ns_q.fit(matrix_small_q, answer) 
    pipe_q = make_pipeline(vectorizer_q, svd_q, ns_q)
    pickle.dump(pipe_q, open('models/TNVED'+str(TNVED)+'.pkl', 'wb'),protocol=4)
   except:
    Errors.append(TNVED)

C:\Users\timka\anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\Users\timka\anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\Users\timka\anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\Users\timka\anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\Users\timka\anaconda3\lib\site-packages\sklearn\decomposition\_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
C:\Us

In [106]:
err2 = []
tr = []
ss=0
for TNVED in TwoClasses:
 try:
  model = pickle.load(open('models/TNVED'+str(TNVED)+'.pkl', 'rb'))
  df = data[data['TNVED2']==TNVED]
  s = model.predict([df.iloc[0]['Post_clean']])
  ss+=1
  if s[0][0]==df.iloc[j]['TNVED4']:
        tr.append(True)
 except:
  err2.append(TNVED)